In [1]:
!pip show tensorflow tensorflow-probability
!pip install tensorflow-probability==0.24.0
!pip install --upgrade keras-tuner
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip -y #Package with useful functions for motor imagery classification based in EEG.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!dir

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras
---
Name: tensorflow-probability
Version: 0.25.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, cloudpickle, decorator, dm-tree, gast, numpy, six
Required-by: dopamine_r

In [3]:
import numpy as np
import os
import itertools
import random
import pickle
import mne
import h5py
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import keras_tuner as kt


from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from scipy.signal import iirnotch, filtfilt, butter, freqz
import matplotlib.pyplot as plt
from scipy.spatial import distance
import networkx as nx
from tqdm import tqdm
from mne.preprocessing import compute_current_source_density
from mne.channels import make_standard_montage, read_custom_montage
from scipy.signal import butter, filtfilt, resample, iirnotch
from gcpds.visualizations.series import plot_eeg
from scipy.stats import norm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from tqdm import tqdm

from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from gcpds.databases import GIGA_MI_ME
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import LeavePGroupsOut, StratifiedGroupKFold
from tensorflow.keras.models import Model
from scipy.spatial.distance import cdist
from sklearn.model_selection import GroupKFold
from tensorflow.keras.models import load_model

from keras_tuner import Objective
from keras_tuner import HyperModel
from keras.layers import Layer, Activation
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.metrics import BinaryAccuracy, Recall

In [5]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4.
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [6]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4 y solo se incluyen las clases 'mano izquierda' (1) y 'mano derecha' (2).
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Filtrar solo las clases de interés (1: mano izquierda, 2: mano derecha)
    clases = [0, 1]
    mask = np.isin(y, clases)
    X = X[mask]
    y = y[mask]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [7]:
@register_keras_serializable(package="CustomLayers")
class TakensConv1D(tf.keras.layers.Layer):
    def __init__(self, dx=5, dy=5, tau=1, mu=4, **kwargs):
        super().__init__(**kwargs)
        self.dx = int(dx)
        self.dy = int(dy)
        self.tau = int(tau)
        self.mu = int(mu)
        self.num_filters = self.dx + self.dy + 1

    def build(self, input_shape):
        kernel_size = self.mu + (self.dx - 1) * self.tau + 1
        kernel_shape = (kernel_size, 1, self.num_filters)
        kernel = tf.zeros(kernel_shape, dtype=tf.float32)

        offsets_x = self.mu + tf.range(self.dx) * self.tau
        offsets_y = tf.range(1, self.dy + 1) * self.tau
        offset_y_t = tf.constant([0], dtype=tf.int32)

        filas_x = tf.range(self.dx)
        filas_y = self.dx + tf.range(self.dy)
        fila_y_t = tf.constant([self.dx + self.dy])

        cols_x = filas_x
        cols_y = filas_y
        col_y_t = fila_y_t

        idx_x = tf.stack([filas_x, offsets_x, cols_x], axis=1)
        idx_y = tf.stack([filas_y, offsets_y, cols_y], axis=1)
        idx_yt = tf.stack([fila_y_t, offset_y_t, col_y_t], axis=1)

        indices_total = tf.concat([idx_x, idx_y, idx_yt], axis=0)
        updates = tf.ones([tf.shape(indices_total)[0]], dtype=tf.float32)

        sort_order = tf.argsort(indices_total[:, 0])
        indices_sorted = tf.gather(indices_total, sort_order)

        row_for_scatter = tf.cast(indices_sorted[:, 1], tf.int32)
        col_for_scatter = tf.cast(indices_sorted[:, 2], tf.int32)

        final_indices = tf.stack([row_for_scatter,
                                  tf.zeros_like(row_for_scatter),
                                  col_for_scatter], axis=1)

        kernel = tf.scatter_nd(final_indices, updates, kernel_shape)
        self.kernel = kernel.numpy()[::-1]

        self.conv1d = tf.keras.layers.Conv1D(
            filters=self.num_filters,
            kernel_size=kernel_size,
            strides=self.tau,
            padding="valid",
            use_bias=False
        )
        self.conv1d.build((None, input_shape[2], 1))
        self.conv1d.set_weights([self.kernel])
        self.conv1d.trainable=False

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        channels = tf.shape(inputs)[1]
        time_steps = tf.shape(inputs)[2]

        reshaped = tf.reshape(inputs, (-1, time_steps, 1))
        conv_output = self.conv1d(reshaped)
        new_time = tf.shape(conv_output)[1]

        output = tf.reshape(conv_output, 
                            (batch_size, channels, new_time, self.num_filters))

        x_sub_t_minus_mu = output[..., :self.dx]
        y_sub_t_minus_1 = output[..., self.dx:self.dx + self.dy]
        y_sub_t = output[..., -1:]

        return x_sub_t_minus_mu, y_sub_t_minus_1, y_sub_t
       

@register_keras_serializable(package="CustomLayers")
class KernelLayer(tf.keras.layers.Layer):
    def __init__(self, 
                 amplitude=1.0,
                 trainable_amplitude=False, 
                 length_scale=1.0,
                 trainable_length_scale=False,
                 alpha=1.0,  # Solo usado para Rational Quadratic
                 trainable_alpha=False,
                 kernel_type="gaussian",  # "gaussian" o "rational_quadratic"
                 **kwargs):
        super(KernelLayer, self).__init__(**kwargs)

        self.init_amplitude = amplitude
        self.trainable_amplitude = trainable_amplitude
        self.init_length_scale = length_scale
        self.trainable_length_scale = trainable_length_scale
        self.init_alpha = alpha
        self.trainable_alpha = trainable_alpha
        self.kernel_type = kernel_type.lower()

    def build(self, input_shape):
        self.amplitude = self.add_weight(
            name="amplitude",
            shape=(),
            initializer=tf.constant_initializer(self.init_amplitude),
            trainable=self.trainable_amplitude,
            dtype=self.dtype
        )

        self.length_scale = self.add_weight(
            name="length_scale",
            shape=(),
            initializer=tf.constant_initializer(self.init_length_scale),
            trainable=self.trainable_length_scale,
            dtype=self.dtype
        )

        if self.kernel_type == "rational_quadratic":
            self.alpha = self.add_weight(
                name="alpha",
                shape=(),
                initializer=tf.constant_initializer(self.init_alpha),
                trainable=self.trainable_alpha,
                dtype=self.dtype
            )

        super(KernelLayer, self).build(input_shape)

    def call(self, X):
        if self.kernel_type == "gaussian":
            kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale
            )
        elif self.kernel_type == "rational_quadratic":
            kernel = tfp.math.psd_kernels.RationalQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale,
                scale_mixture_rate=self.alpha
            )
        else:
            raise ValueError(f"Unsupported kernel_type: {self.kernel_type}")
        
        return kernel.matrix(X, X)
       
@register_keras_serializable(package="CustomLayers")
class TransferEntropyLayer(tf.keras.layers.Layer):
    def __init__(self, alpha=2, **kwargs):

        super().__init__(**kwargs)
        self.alpha = int(alpha)

    def compute_entropy(self, K_hadamard):
        
        trace_hadamard = tf.reduce_sum(tf.linalg.diag_part(K_hadamard), axis=-1) 
        trace_hadamard = tf.expand_dims(tf.expand_dims(trace_hadamard, axis=-1), axis=-1)
        K_normalized = K_hadamard / trace_hadamard

        K_power = tf.linalg.matmul( K_normalized , K_normalized, grad_a=True, grad_b=True ) 
        trace_power = tf.reduce_sum(tf.linalg.diag_part(K_power), axis=-1)  
        H_alpha = (1 / (1 - self.alpha)) * tf.math.log(trace_power)
        return H_alpha

    def call(self, K_x, K_y_minus_1, K_y):
        
        K_x_exp = tf.expand_dims(K_x, axis=2)
        
        K_y_minus_1_exp = tf.expand_dims(K_y_minus_1, axis=1)
        K_y_exp = tf.expand_dims(K_y, axis=1)

        
        H_1 = self.compute_entropy(K_y_minus_1_exp * K_x_exp)
        H_2 = self.compute_entropy(K_y_exp * K_y_minus_1_exp * K_x_exp)
        H_3 = self.compute_entropy(K_y_exp * K_y_minus_1_exp)
        H_4 = self.compute_entropy(K_y_minus_1_exp)

        TE = H_1 - H_2 + H_3 - H_4
        
        return TE
        
@register_keras_serializable(package="CustomLayers")      
class RemoveDiagonalFlatten(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)

    def call(self, inputs):
        
        shape_dyn = tf.shape(inputs)  
        batch_size = shape_dyn[0]
        c = tf.shape(inputs)[1]

        
        tf.debugging.assert_equal(
            tf.shape(inputs)[1], tf.shape(inputs)[2],
            message="RemoveDiagonalFlatten: la matriz de entrada no es cuadrada."
        )  
        diag_mask = tf.eye(c, dtype=inputs.dtype)  
        inputs_no_diag = inputs * (1 - diag_mask) 
        flattened = tf.reshape(inputs_no_diag, [batch_size, -1])  
        non_diag = tf.boolean_mask(flattened, tf.reshape(1 - diag_mask, [-1]), axis=1)
        num_features = c * (c - 1)  
        result = tf.reshape(non_diag, [batch_size, num_features])  

        return result

In [8]:
from tensorflow.keras.metrics import Metric
@register_keras_serializable(package="CustomMetrics")
class Sensitivity(Metric):
    def __init__(self, name='sensitivity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Accuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        num = self.tp.result() + self.tn.result()
        den = num + self.fp.result() + self.fn.result() + tf.keras.backend.epsilon()
        return num / den

    def reset_states(self):
        self.tp.reset_states()
        self.tn.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.tp.result() / (self.tp.result() + self.fp.result() + tf.keras.backend.epsilon())
        recall = self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Specificity(Metric):
    def __init__(self, name='specificity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tn.result() / (self.tn.result() + self.fp.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tn.reset_states()
        self.fp.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [9]:
# Crear instancia del dataset
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0 

# Cargar los datos del sujeto en modo 'training'
X, y = load_BCICIV2a(db, sbj=5, mode='training', fs=fs)
print(f'tamaño de X:', X.shape)
print(f'tamaño de y:', y.shape)

tamaño de X: (129, 22, 500)
tamaño de y: (129,)


In [8]:
train_data, val_data, train_labels, val_labels = train_test_split(
    X, y,
    test_size=0.2,
    random_state=72,
    stratify=y
)

def create_tunable_model(dx, dy, tau, mu,
                         kernel_type, learning_rate,
                         kernel_size):
    input_eeg = tf.keras.Input(shape=(22, 500), name='input_eeg')

    # Bloque 1 con kernel_size buscable
    x = tf.keras.layers.DepthwiseConv1D(
        kernel_size=kernel_size, 
        strides=1,
        padding='valid',
        activation='relu',
        depth_multiplier=1,
        data_format="channels_first",
        name='block1_depthwise_conv1d'
    )(input_eeg)

    x = tf.keras.layers.AveragePooling1D(
        pool_size=4,
        strides=4,
        padding='valid',
        data_format="channels_first",
        name='block1_avg_pooling'
    )(x)

    x = tf.keras.layers.BatchNormalization(
        axis=1,
        name='block1_batch_norm'
    )(x)

    # Bloque 2: TakensConv1D
    takens = TakensConv1D(dx=dx, dy=dy, tau=tau, mu=mu,
                          name='takens_conv1d')(x)
    x_sub, y_minus_1, y_t = takens

    # Proyecciones densas
    x_sub     = tf.keras.layers.Dense(dx, activation=None,
                                       use_bias=False,
                                       name='dense_proj_x')(x_sub)
    y_minus_1 = tf.keras.layers.Dense(dy, activation=None,
                                       use_bias=False,
                                       name='dense_proj_y_1')(y_minus_1)
    y_t       = tf.keras.layers.Dense(1, activation=None,
                                       use_bias=False,
                                       name='dense_proj_y')(y_t)

    # Kernel layers fijos
    def fixed_kernel(name):
        layer = KernelLayer(
            amplitude=1.0, trainable_amplitude=False,
            length_scale=1.0, trainable_length_scale=False,
            alpha=1.0, trainable_alpha=False,
            kernel_type=kernel_type,
            name=name
        )
        layer.trainable = False
        return layer

    Kx  = fixed_kernel('kernel_x')(x_sub)
    Ky1 = fixed_kernel('kernel_y_minus_1')(y_minus_1)
    Ky  = fixed_kernel('kernel_y')(y_t)

    # Transferencia de entropía + aplanado
    TE   = TransferEntropyLayer(alpha=2,
                                name='transfer_entropy')(Kx, Ky1, Ky)
    flat = RemoveDiagonalFlatten(name='remove_diag_flatten')(TE)

    # Salida
    h   = tf.keras.layers.Dense(10, activation='relu',
                                name='dense_1')(flat)
    out = tf.keras.layers.Dense(1, activation='sigmoid',
                                name='output')(h)

    model = tf.keras.Model(inputs=input_eeg, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            BinaryAccuracy(name='accuracy'),
            F1Score(name='f1_score'),
            Recall(name='sensitivity'),
            Specificity(name='specificity')
        ]
    )
    return model

def build_model(hp):
    # Hiperparámetros a buscar
    dx          = hp.Int('dx',    min_value=1,  max_value=10, step=1)
    dy          = hp.Int('dy',    min_value=1,  max_value=10, step=1)
    tau         = hp.Int('tau',   min_value=1,  max_value=5,  step=1)
    mu          = hp.Int('mu',    min_value=0,  max_value=10, step=1)
    lr          = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    kernel_type = 'rational_quadratic'

    # Nuevo: rango buscable para kernel_size
    kernel_size = hp.Int(
        'kernel_size',
        min_value=3,      # ventanas muy pequeñas (3 muestras)
        max_value=125,    # hasta ~125 muestras (~0.25 s a 500 Hz)
        step=2            # sólo impares
    )

    # Validación de que el embedding window quepa tras Conv+Pool
    conv_len = 500 - kernel_size + 1
    pool_len = (conv_len - 4) // 4 + 1
    window   = mu + (dx - 1) * tau + 1
    if window > pool_len:
        # Trial inválido: modelo trivial
        m = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        m.compile('adam', 'binary_crossentropy', ['accuracy'])
        return m

    # Construcción del modelo real
    return create_tunable_model(
        dx, dy, tau, mu,
        kernel_type, lr,
        kernel_size
    )

# Configuración del tuner
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective=kt.Objective("val_f1_score", direction="max"),
    max_trials=50,
    executions_per_trial=2,
    directory="tuner_dir5",
    project_name="takens_te_tuning_5"
)

stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=1
)

# Inicio de la búsqueda
tuner.search(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=200,
    callbacks=[stop_early]
)

# Extracción del mejor resultado
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_score = best_trial.score  # val_f1_score

# Guardado en DataFrame y CSV
import pandas as pd

best_dict = {
    'dx':             best_hp.get('dx'),
    'dy':             best_hp.get('dy'),
    'tau':            best_hp.get('tau'),
    'mu':             best_hp.get('mu'),
    'kernel_size':    best_hp.get('kernel_size'),
    'learning_rate':  best_hp.get('learning_rate'),
    'val_f1_score':   best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)
df_best.to_csv('best_hyperparameters_with_score5.csv', index=False)

Trial 50 Complete [00h 01m 00s]
val_f1_score: 0.6754385530948639

Best val_f1_score So Far: 0.7510775327682495
Total elapsed time: 00h 49m 39s
   dx  dy  tau  mu  kernel_size  learning_rate  val_f1_score
0   6   1    3   6          123           0.01      0.751078


In [9]:
# 1) Obtener el modelo ya entrenado con los mejores hiperparámetros
best_model = tuner.get_best_models(num_models=1)[0]

# 2) Guardarlo en /kaggle/working
best_model.save("best_hp_model5.keras") 


# Crear instancia del dataset
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0 

# Cargar los datos del sujeto en modo 'training'
X, y = load_BCICIV2a(db, sbj=5, mode='evaluation', fs=fs)
print(f'tamaño de X:', X.shape)
print(f'tamaño de X:', y.shape)

results = best_model.evaluate(X, y, return_dict=True)

print("Resultados evaluación:")
print(f"  Accuracy:     {results['accuracy']:.4f}")
print(f"  F1 Score:     {results['f1_score']:.4f}")
print(f"  Loss:         {results['loss']:.4f}")
print(f"  Sensitivity:  {results['sensitivity']:.4f}")
print(f"  Specificity:  {results['specificity']:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


tamaño de X: (135, 22, 500)
tamaño de X: (135,)
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 487ms/step - accuracy: 0.4892 - f1_score: 0.5474 - loss: 0.7084 - sensitivity: 0.7123 - specificity: 0.3188
Resultados evaluación:
  Accuracy:     0.4963
  F1 Score:     0.5641
  Loss:         0.7071
  Sensitivity:  0.6769
  Specificity:  0.3286


In [10]:
# 1) Recuperar los mejores HyperParameters y Trial
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

# 2) Extraer puntuación de validación (F1)
best_score = best_trial.score  # corresponde a val_f1_score

# 3) Construir DataFrame con todos los valores
import pandas as pd

best_dict = {
    'dx':             best_hp.get('dx'),
    'dy':             best_hp.get('dy'),
    'tau':            best_hp.get('tau'),
    'mu':             best_hp.get('mu'),
    'kernel_size':    best_hp.get('kernel_size'),
    'learning_rate':  best_hp.get('learning_rate'), 
    'val_f1_score':   best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)

# 4) Guardar a CSV
df_best.to_csv('best_hyperparameters_with_score5.csv', index=False)

   dx  dy  tau  mu  kernel_size  learning_rate  val_f1_score
0   6   1    3   6          123           0.01      0.751078


In [11]:
# 1) Recorremos todos los trials guardados en el tuner
records = []
for trial in tuner.oracle.trials.values():
    # Cada trial tiene un objeto HyperParameters donde .values es un dict de {hp_name: value}
    hp_dict = trial.hyperparameters.values.copy()
    # Añadimos el score (val_f1_score) de este trial
    hp_dict['val_f1_score'] = trial.score
    # Opcional: identifica el trial
    hp_dict['trial_id'] = trial.trial_id
    records.append(hp_dict)

# 2) Creamos el DataFrame
df_all = pd.DataFrame(records)

# 3) Ordenamos por la métrica descendente para ver primero los mejores
df_all = df_all.sort_values('val_f1_score', ascending=False).reset_index(drop=True)

# 4) Mostramos y guardamos
print(df_all)
df_all.to_csv('all_trials_hyperparameters5.csv', index=False)

    dx  dy  tau  mu  learning_rate  kernel_size  val_f1_score trial_id
0    6   1    3   6         0.0100          123      0.751078       03
1   10   1    1   1         0.0010          117      0.739583       30
2    8   1    1   0         0.0010           93      0.718264       34
3    7   1    1   0         0.0100          125      0.717857       18
4    1   6    1   2         0.0100          125      0.715950       38
5    1   4    2   3         0.0100           99      0.708333       04
6    7   2    1   2         0.0010          119      0.704762       28
7    8  10    3  10         0.0100           75      0.696970       05
8    4   3    1   3         0.0100           99      0.696970       26
9    8  10    4   5         0.0100           91      0.694444       00
10   7   2    1   1         0.0100           91      0.693333       27
11   1   5    1   4         0.0100           71      0.688172       14
12  10   1    1   2         0.0001          103      0.684685       33
13  10

# Entrenando con Quadratic

In [13]:
model = load_model("/kaggle/working/best_hp_model5.keras", compile=True)

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=100,
    restore_best_weights=True,
    verbose=1
)
csv_logger = CSVLogger("continued_training.log", append=True)

# 3) Carga los datos de BCI Competition IV-2a
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0

#    – Split de entrenamiento
X_full, y_full = load_BCICIV2a(db, sbj=5, mode='training',   fs=fs)
#    – Split de evaluación (test final)
X_test, y_test = load_BCICIV2a(db, sbj=5, mode='evaluation', fs=fs)

# 4) Divide el set de entrenamiento en train/val (80/20)
train_data, val_data, train_labels, val_labels = train_test_split(
    X_full, y_full,
    test_size=0.2,
    random_state=42,
    stratify=y_full
)

print(f"Train: {train_data.shape}, {train_labels.shape}")
print(f"Val:   {val_data.shape}, {val_labels.shape}")
print(f"Test:  {X_test.shape}, {y_test.shape}")

# 5) Continúa el entrenamiento
history = model.fit(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=1000,
    callbacks=[early_stopping, csv_logger]
)

# 6) Evalúa en el conjunto de evaluación
results = model.evaluate(X_test, y_test, return_dict=True)
print("Resultados en test:")
for name, value in results.items():
    print(f"  {name}: {value:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Train: (103, 22, 500), (103,)
Val:   (26, 22, 500), (26,)
Test:  (135, 22, 500), (135,)
Epoch 1/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.5649 - f1_score: 0.6549 - loss: 0.6921 - sensitivity: 0.7568 - specificity: 0.3625 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.7051 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 2/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7483 - f1_score: 0.7350 - loss: 0.6625 - sensitivity: 0.6749 - specificity: 0.8286 - val_accuracy: 0.6538 - val_f1_score: 0.6667 - val_loss: 0.6774 - val_sensitivity: 0.6923 - val_specificity: 0.6154
Epoch 3/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7589 - f1_score: 0.6492 - loss: 0.6541 - sensitivity: 0.4891 - specificity: 0.9920 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.9181 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 4/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6902 - f1_score: 0.7779 - loss: 0.6163 - sens

# Validación cruzada 5 Folds 

In [14]:
# 1) Parámetros generales
N_SPLITS = 5
RANDOM_STATE = 42
EPOCHS = 1000
MODEL_PATH = "/kaggle/working/best_hp_model5.keras"

# 2) Función para crear callbacks de cada fold
def get_callbacks(fold):
    return [
        EarlyStopping(
            monitor="val_loss", mode="min", patience=100,
            restore_best_weights=True, verbose=1
        ),
        CSVLogger(f"continued_training_fold{fold}.log", append=True),
        ModelCheckpoint(
            filepath=f"best_model_fold{fold}.keras",
            monitor="val_loss",               # o cambia a "val_accuracy" u otra métrica
            mode="min",                       # "min" si monitoreas pérdidas, "max" si monitoreas accuracy/F1
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
    ]

# 3) Carga de datos
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0
X_full, y_full = load_BCICIV2a(db, sbj=5, mode='training',   fs=fs)
X_test, y_test = load_BCICIV2a(db, sbj=5, mode='evaluation', fs=fs)

print(f"Full train set: {X_full.shape}, {y_full.shape}")
print(f"Evaluation set: {X_test.shape}, {y_test.shape}")

# 4) Cross‐validation estratificada
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

# 5) Loop de folds
fold_results = []
for fold, (train_idx, val_idx) in enumerate(skf.split(X_full, y_full), start=1):
    print(f"\n\n=== Fold {fold}/{N_SPLITS} ===")
    # 5.1) Carga modelo fresco con los pesos previos
    model = load_model(MODEL_PATH, compile=True)

    # 5.2) Split train/val para este fold
    X_train, y_train = X_full[train_idx], y_full[train_idx]
    X_val,   y_val   = X_full[val_idx],   y_full[val_idx]
    print(f"  Train: {X_train.shape}, {y_train.shape}")
    print(f"  Val:   {X_val.shape},   {y_val.shape}")

    # 5.3) Entrena y guarda el mejor modelo del fold
    history = model.fit(
        x=X_train, y=y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=get_callbacks(fold),
        verbose=2
    )

    # 5.4) Carga el mejor modelo guardado antes de evaluar
    best_model = load_model(f"best_model_fold{fold}.keras", compile=True)
    results = best_model.evaluate(X_test, y_test, return_dict=True, verbose=0)
    print(f"  Eval (best model fold{fold}): " +
          ", ".join(f"{k}={v:.4f}" for k, v in results.items()))

    fold_results.append(results)

# 6) Resumen de performance promedio
metrics = fold_results[0].keys()
print("\n=== Resumen Cross‐Validation ===")
for m in metrics:
    vals = [r[m] for r in fold_results]
    print(f"{m:12s}: {np.mean(vals):.4f} ± {np.std(vals, ddof=1):.4f}")

Full train set: (129, 22, 500), (129,)
Evaluation set: (135, 22, 500), (135,)


=== Fold 1/5 ===
  Train: (103, 22, 500), (103,)
  Val:   (26, 22, 500),   (26,)
Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Epoch 1: val_loss improved from inf to 0.83920, saving model to best_model_fold1.keras
4/4 - 17s - 4s/step - accuracy: 0.5825 - f1_score: 0.6667 - loss: 0.6837 - sensitivity: 0.8113 - specificity: 0.3400 - val_accuracy: 0.5000 - val_f1_score: 0.0000e+00 - val_loss: 0.8392 - val_sensitivity: 0.0000e+00 - val_specificity: 1.0000
Epoch 2/1000

Epoch 2: val_loss improved from 0.83920 to 0.72496, saving model to best_model_fold1.keras
4/4 - 0s - 58ms/step - accuracy: 0.5631 - f1_score: 0.3077 - loss: 0.6554 - sensitivity: 0.1887 - specificity: 0.9600 - val_accuracy: 0.5000 - val_f1_score: 0.6486 - val_loss: 0.7250 - val_sensitivity: 0.9231 - val_specificity: 0.0769
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.72496
4/4 - 0s - 43ms/step - accuracy: 0.7961 - f1_score: 0.8264 - loss: 0.5888 - sensitivity: 0.9434 - specificity: 0.6400 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.8793 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 4/1000

Epoch 4: val_los

In [11]:
# --- Parámetros ---
N_SPLITS     = 5
RANDOM_STATE = 42
EPOCHS       = 1000
FS           = 250.0
MODEL_PATH   = "/kaggle/input/best-models-sbjte/best_hp_model5.keras"

# Métricas a trackear
metrics = ['accuracy', 'specificity', 'sensitivity', 'f1_score']

# Diccionarios para acumular resultados
train_eval = {m: [] for m in metrics}
val_eval   = {m: [] for m in metrics}
test_eval  = {m: [] for m in metrics}

# Callbacks que usamos en cada fold
def get_callbacks(fold):
    return [
        EarlyStopping(
            monitor="val_f1_score",
            mode="max",
            patience=100,
            restore_best_weights=True,
            verbose=1
        ),
        CSVLogger(f"continued_training_fold{fold}.log", append=True),
        ModelCheckpoint(
            filepath=f"best_model_fold{fold}.keras",
            monitor="val_accuracy",
            mode="max",
            save_best_only=True,
            verbose=1
        ),
    ]

# --- Carga de datos ---
db = Dataset_2a('/kaggle/input/dataset-2a')
X_full, y_full = load_BCICIV2a(db, sbj=5, mode='training',   fs=FS)
X_test, y_test = load_BCICIV2a(db, sbj=5, mode='evaluation', fs=FS)

print(f"Training set:   {X_full.shape}, {y_full.shape}")
print(f"Evaluation set: {X_test.shape},  {y_test.shape}")

# Preparamos el Stratified K-Fold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_full, y_full), start=1):
    print(f"\n=== Fold {fold}/{N_SPLITS} ===")
    X_train, y_train = X_full[train_idx], y_full[train_idx]
    X_val,   y_val   = X_full[val_idx],   y_full[val_idx]

    # 1) Cargar modelo con pesos y compilación original
    model = load_model(MODEL_PATH, compile=True)

    # 2) Continuar entrenamiento en este fold
    model.fit(
        x=X_train, y=y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=get_callbacks(fold),
        verbose=2
    )
# /kaggle/input/cross-validation/best_model_fold1.keras
    # 3) Cargar el mejor modelo guardado por ModelCheckpoint
    best_model = load_model(f"/kaggle/input/cross-validation/best_model_fold{fold}.keras", compile=True)

    # 4) Evaluar en train, validación y evaluación final
    res_tr   = best_model.evaluate(X_train, y_train, return_dict=True, verbose=0)
    res_va   = best_model.evaluate(X_val,   y_val,   return_dict=True, verbose=0)
    res_test = best_model.evaluate(X_test,  y_test,  return_dict=True, verbose=0)

    # 5) Almacenar cada métrica
    for m in metrics:
        train_eval[m].append( res_tr[m] )
        val_eval[m].append(   res_va[m] )
        test_eval[m].append(  res_test[m] )

# --- Función para mostrar resumen ---
def summarize(eval_dict, title):
    print(f"\n--- {title} ---")
    for m, vals in eval_dict.items():
        mean = np.mean(vals)
        std  = np.std(vals, ddof=1)
        print(f"{m:12s}: {mean:.4f} ± {std:.4f}")

# Imprimimos los promedios ± desviaciones estándar
summarize(train_eval, "TRAIN   (best model por fold)")
summarize(val_eval,   "VALIDATION (best model por fold)")
summarize(test_eval,  "EVALUATION  (best model por fold)")

Training set:   (129, 22, 500), (129,)
Evaluation set: (135, 22, 500),  (135,)

=== Fold 1/5 ===
Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
W0000 00:00:1750365996.933563     248 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366002.417528     248 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366008.594575     247 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert



Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model_fold1.keras
4/4 - 18s - 5s/step - accuracy: 0.5437 - f1_score: 0.5983 - loss: 0.6891 - sensitivity: 0.6604 - specificity: 0.4200 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.7040 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 2/1000

Epoch 2: val_accuracy did not improve from 0.50000
4/4 - 0s - 42ms/step - accuracy: 0.8350 - f1_score: 0.8522 - loss: 0.6284 - sensitivity: 0.9245 - specificity: 0.7400 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 1.1915 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 3/1000

Epoch 3: val_accuracy did not improve from 0.50000
4/4 - 0s - 43ms/step - accuracy: 0.5922 - f1_score: 0.7162 - loss: 0.5999 - sensitivity: 1.0000 - specificity: 0.1600 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.9170 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 4/1000

Epoch 4: val_accuracy improved from 0.50000 to 

W0000 00:00:1750366030.403874     246 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366032.455913     247 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366034.448354     247 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert



=== Fold 2/5 ===
Epoch 1/1000


...


=== Fold 3/5 ===
Epoch 1/1000


W0000 00:00:1750366087.275881     245 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366092.662393     245 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366098.508836     245 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert



Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model_fold3.keras
4/4 - 18s - 4s/step - accuracy: 0.5340 - f1_score: 0.6308 - loss: 0.6875 - sensitivity: 0.7736 - specificity: 0.2800 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 0.8737 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 2/1000

Epoch 2: val_accuracy did not improve from 0.50000
4/4 - 0s - 42ms/step - accuracy: 0.6117 - f1_score: 0.7260 - loss: 0.6544 - sensitivity: 1.0000 - specificity: 0.2000 - val_accuracy: 0.5000 - val_f1_score: 0.6667 - val_loss: 1.3621 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 3/1000

Epoch 3: val_accuracy did not improve from 0.50000
4/4 - 0s - 40ms/step - accuracy: 0.6796 - f1_score: 0.7591 - loss: 0.6094 - sensitivity: 0.9811 - specificity: 0.3600 - val_accuracy: 0.4231 - val_f1_score: 0.5714 - val_loss: 0.8616 - val_sensitivity: 0.7692 - val_specificity: 0.0769
Epoch 4/1000

Epoch 4: val_accuracy did not improve from 0.50000


W0000 00:00:1750366119.705244     245 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert
W0000 00:00:1750366121.775166     245 assert_op.cc:38] Ignoring Assert operator functional_1/remove_diag_flatten_1/assert_equal_1/Assert/Assert



=== Fold 4/5 ===
Epoch 1/1000

Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to best_model_fold4.keras
4/4 - 18s - 5s/step - accuracy: 0.6311 - f1_score: 0.6935 - loss: 0.7051 - sensitivity: 0.8269 - specificity: 0.4314 - val_accuracy: 0.5385 - val_f1_score: 0.7000 - val_loss: 0.7654 - val_sensitivity: 1.0000 - val_specificity: 0.0000e+00
Epoch 2/1000

Epoch 2: val_accuracy improved from 0.53846 to 0.65385, saving model to best_model_fold4.keras
4/4 - 0s - 57ms/step - accuracy: 0.7670 - f1_score: 0.7551 - loss: 0.6359 - sensitivity: 0.7115 - specificity: 0.8235 - val_accuracy: 0.6538 - val_f1_score: 0.6897 - val_loss: 0.6747 - val_sensitivity: 0.7143 - val_specificity: 0.5833
Epoch 3/1000

Epoch 3: val_accuracy did not improve from 0.65385
4/4 - 0s - 42ms/step - accuracy: 0.8252 - f1_score: 0.8364 - loss: 0.6075 - sensitivity: 0.8846 - specificity: 0.7647 - val_accuracy: 0.5385 - val_f1_score: 0.7000 - val_loss: 0.7818 - val_sensitivity: 1.0000 - val_specificity: 0